<!--NOTEBOOK_HEADER-->
*This notebook contains material from [CBE60499](https://ndcbe.github.io/CBE60499);
content is available [on Github](git@github.com:ndcbe/CBE60499.git).*


<!--NAVIGATION-->
< [2.9 Supplementary material: data for parmest tutorial](https://ndcbe.github.io/CBE60499/02.09-Parmest-generate-data.html) | [Contents](toc.html) | [Tag Index](tag_index.html) | [2.11 Pyomo Homework 2](https://ndcbe.github.io/CBE60499/02.11-Pyomo2.html) ><p><a href="https://colab.research.google.com/github/ndcbe/CBE60499/blob/master/docs/02.10-Pyomo1.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://ndcbe.github.io/CBE60499/02.10-Pyomo1.ipynb"> <img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>

In [ ]:
# IMPORT DATA FILES USED BY THIS NOTEBOOK
import os,  requests

file_links = [("data/knapsack_data.csv", "https://ndcbe.github.io/CBE60499/data/knapsack_data.csv")]

# This cell has been added by nbpages. Run this cell to download data files required for this notebook.

for filepath, fileurl in file_links:
    stem, filename = os.path.split(filepath)
    if stem:
        if not os.path.exists(stem):
            os.mkdir(stem)
    if not os.path.isfile(filepath):
        with open(filepath, 'wb') as f:
            response = requests.get(fileurl)
            f.write(response.content)


# 2.10 Pyomo Homework 1

**Due Date:** 2/16/2021

In [ ]:
# This code cell installs packages on Colab

import sys
if "google.colab" in sys.modules:
    !wget "https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py"
    import helper
    helper.install_idaes()
    helper.install_ipopt()
    helper.install_glpk()
    helper.download_data(['knapsack_data.xlsx','knapsack_data.csv'])

In [ ]:
## IMPORT LIBRARIES
from pyomo.environ import *
import pandas as pd

Special thanks to the Pyomo team for create these excercises as part of their excellent PyomoFest workshop.

## 2.10.1 Pyomo Fundamentals

### 2.10.1.1 Knapsack example

Solve the knapsack problem given below using GLPK and answer the following questions:

1. Which items are acquired in the optimal solution?

2. What is the value of the selected items?

In [ ]:
A = ['hammer', 'wrench', 'screwdriver', 'towel']
b = {'hammer':8, 'wrench':3, 'screwdriver':6, 'towel':11}
w = {'hammer':5, 'wrench':7, 'screwdriver':4, 'towel':3}
W_max = 14

model = ConcreteModel()
model.x = Var( A, within=Binary )

model.obj = Objective(
    expr = sum( b[i]*model.x[i] for i in A ), 
    sense = maximize )

model.weight_con = Constraint(
    expr = sum( w[i]*model.x[i] for i in A ) <= W_max )

# YOUR SOLUTION HERE

model.display()

**Question Answers**

1. *Fill in here*

2. *Fill in here*

### 2.10.1.2 Knapsack example with improve printing

Complete the missing lines in the code below to produce formatted output: print the total weight, the value of the items selected (the objective), and the items acquired in the optimal solution. Note, the Pyomo value function should be used to get the floating point value of Pyomo modeling components (e.g., print(value(model.x[i])).

In [ ]:
A = ['hammer', 'wrench', 'screwdriver', 'towel']
b = {'hammer':8, 'wrench':3, 'screwdriver':6, 'towel':11}
w = {'hammer':5, 'wrench':7, 'screwdriver':4, 'towel':3}
W_max = 14

model = ConcreteModel()
model.x = Var( A, within=Binary )

model.obj = Objective(
    expr = sum( b[i]*model.x[i] for i in A ), 
    sense = maximize )

model.weight_con = Constraint(
    expr = sum( w[i]*model.x[i] for i in A ) <= W_max )

opt = SolverFactory('glpk')
opt_success = opt.solve(model)

total_weight = sum( w[i]*value(model.x[i]) for i in A )
# YOUR SOLUTION HERE
print('-------------------------')

### 2.10.1.3 Changing data

Using your code from **Question 1.2**, if we were to increase the value of the wrench, at what point would it become selected as part of the optimal solution?

In [ ]:
# YOUR SOLUTION HERE

**Question Answer**

*Fill in here*

### 2.10.1.4 Loading data from Excel

In the code above, the data is hardcoded at the top of the file. Instead of hardcoding the data, use Python to load the data from a difference source. You may use Pandas to load data from 'knapsack_data.xlsx' into a dataframe. You will then need to write code to obtain a dictionary from the dataframe.

In [ ]:
df_items = pd.read_excel('./data/knapsack_data.xlsx', sheet_name='data', header=0, index_col=0)
W_max = 14

A = df_items.index.tolist()
# YOUR SOLUTION HERE

model = ConcreteModel()
model.x = Var( A, within=Binary )

model.obj = Objective(
    expr = sum( b[i]*model.x[i] for i in A ), 
    sense = maximize )

model.weight_con = Constraint(
    expr = sum( w[i]*model.x[i] for i in A ) <= W_max )

opt = SolverFactory('glpk')
opt_success = opt.solve(model)

total_weight = sum( w[i]*value(model.x[i]) for i in A )
print('Total Weight:', total_weight)
print('Total Benefit:', value(model.obj))

print('%12s %12s' % ('Item', 'Selected'))
print('=========================')
for i in A:
    acquired = 'No'
    if value(model.x[i]) >= 0.5:
        acquired = 'Yes'
    print('%12s %12s' % (i, acquired))
print('-------------------------')

### 2.10.1.5 NLP vs. MIP

Solve the knapsack problem with IPOPT instead of glpk. Print the solution values for model.x. What happened? Why?

*Hint*: Switch `glpk` to `ipopt` in the call to `SolverFactory`.

In [ ]:
from pyomo.environ import *

A = ['hammer', 'wrench', 'screwdriver', 'towel']
b = {'hammer':8, 'wrench':3, 'screwdriver':6, 'towel':11}
w = {'hammer':5, 'wrench':7, 'screwdriver':4, 'towel':3}
W_max = 14

model = ConcreteModel()
model.x = Var( A, within=Binary )

model.obj = Objective(
    expr = sum( b[i]*model.x[i] for i in A ), 
    sense = maximize )

model.weight_con = Constraint(
    expr = sum( w[i]*model.x[i] for i in A ) <= W_max )

# YOUR SOLUTION HERE
opt_success = opt.solve(model)

model.pprint()

**Question Answers**

*Fill in here*

## 2.10.2 More Pyomo Fundamentals

### 2.10.2.1 Knapsack problem with rules

Rules are important for defining in-dexed constraints, however, they can also be used for single (i.e. scalar) constraints. Reimplement the knapsack model from **Question 1.1** using rules for the objective and the constraints.

In [ ]:
# YOUR SOLUTION HERE

### 2.10.2.2 Integer formulation of the knapsack problem

Consider again the knapsack problem. Assume now that we can acquire multiple items of the same type. In this new formulation, $x_i$ is now an integer variable instead of a binary variable. One way to formulate this problem is as follows:


$\max_{q,x} \sum_{i\in{A}}v_ix_i$

s.t. $\sum_{i\in{A}}w_ix_i\leq W_{max}$

$x_i=\sum_{j=0}^Njq_{i,j}, \forall i \in{A}$

$0\leq x\leq N$

$q_{i,j} \in {0,1}, \forall i \in A, j \in {0...N}$

Starting with your code from **Question 2.1**, implement this new formulation and solve. Is the solution surprising?

In [ ]:
# YOUR SOLUTION HERE

**Question Answer**

*Fill in here*

<!--NAVIGATION-->
< [2.9 Supplementary material: data for parmest tutorial](https://ndcbe.github.io/CBE60499/02.09-Parmest-generate-data.html) | [Contents](toc.html) | [Tag Index](tag_index.html) | [2.11 Pyomo Homework 2](https://ndcbe.github.io/CBE60499/02.11-Pyomo2.html) ><p><a href="https://colab.research.google.com/github/ndcbe/CBE60499/blob/master/docs/02.10-Pyomo1.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://ndcbe.github.io/CBE60499/02.10-Pyomo1.ipynb"> <img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>